In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
sales = pd.read_csv('sales_mod.csv')

/tmp/ipykernel_1976943/1754691465.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv('sales_mod.csv')


In [3]:
sales.buyer_name = sales.buyer_name.str.encode('ascii', 'ignore').str.decode('ascii')
sales.seller_name = sales.seller_name.str.encode('ascii', 'ignore').str.decode('ascii')
sales.buyer_name = sales.buyer_name.str.replace(r'[^a-zA-Z0-9\-]', ' ', regex=True).str.strip()
sales.seller_name = sales.seller_name.str.replace(r'[^a-zA-Z0-9\-]', ' ', regex=True).str.strip()

In [4]:
entity_keywords = r"llc| ll$| l$|l l c|estate|training|construction|building|masonry|apartments|plumbing|service|professional|roofing|advanced|office|\blaw\b|loan|legal|production|woodwork|concepts|corp| company| united|\binc\b|county|entertainment|community|heating|cooling|partners|equity|indsutries|series|revitalization|collection|agency|renovation|consulting|flippers|estates|\bthe \b|dept|funding|opportunity|improvements|servicing|equities|sale|judicial| in$|bank|\btrust\b|holding|investment|housing|properties|limited|realty|development|capital|management|developers|construction|rentals|group|investments|invest|residences|enterprise|enterprises|ventures|remodeling|specialists|homes|business|venture|restoration|renovations|maintenance|ltd|real estate|builders|buyers|property|financial|associates|consultants|international|acquisitions|credit|design|homeownership|solutions|home|diversified|assets|family|land|revocable|services|rehabbing|living|county of cook|fannie mae|land|veteran|mortgage|savings|lp$"

def get_id(row, col: str):

    column = col + '_name'
    words = str(row[column]).lower()

    words = re.sub(r' amp ','', words)
    words = re.sub(' +', ' ', words)

    if words.isspace() or re.search(r'^[.]*$', words):
        id = 'Empty Name'
        return id

    if any(x in words for x in ['vt investment corpor', 'v t investment corp']):
        return 'vt investment corporation'

    if any(x in words for x in ['first integrity group inc', 'first integrity group in']):
        return 'first integrity group inc'

    if words in ['deutsche bank national tr']:
        return 'deutsche bank national trust company'

    if any(x in words for x in ['cirrus investment group l', 'cirrus investment group']):
        return 'cirrus investment group'

    if any(x in words for x in ['fannie mae aka federal na',  'fannie mae a k a federal', 'federal national mortgage']):
        return 'fannie mae'

    if any(x in words for x in ['the judicial sales corpor', 'judicial sales corp', 'judicial sales corporatio', 'judicial sale corp', 'the judicial sales corp']):
        return 'the judicial sales corporation'

    if any(x in words for x in ['jpmorgan chase bank n a', 'jpmorgan chase bank nati']):
        return 'jp morgan chase bank'

    if any(x in words for x in ['wells fargo bank na',  'wells fargo bank n a',  'wells fargo bank nationa',  'wells fargo bank n a a']):
        return 'wells fargo bank national'

    if any(x in words for x in ['bayview loan servicing l',  'bayview loan servicing ll']):
        return 'bayview loan servicing llc'

    if any(x in words for x in ['thr property illinois l', 'thr property illinois lp']):
        return 'thr property illinois lp'

    if any(x in words for x in ['ih3 property illinois lp', 'ih3 property illinois l']):
        return 'ih3 property illinois lp'

    if any(x in words for x in ['ih2 property illinois lp', 'ih2 property illinois l']):
        return 'ih2 property illinois lp'

    if any(x in words for x in ['secretary of housing and',  'the secretary of housing', 'secretary of housing ']):
        return 'secretary of housing and urban development'

    if any(x in words for x in ['secretary of veterans aff', 'the secretary of veterans']):
        return 'secretary of veterans affairs'

    if any(x in words for x in ['bank of america n a', 'bank of america na', 'bank of america national',]):
        return 'bank of america national'

    if any(x in words for x in ['us bank national association', 'u s bank national assoc', 'u s bank national associ', 'u s bank trust n a as', 'u s bank n a', 'us bank national associat', 'u s bank trust national']):
        return 'us bank national association'

    words = re.sub('suc t$|as succ t$|successor tr$|successor tru$|successor trus$|successor trust$|successor truste$|successor trustee$|successor t$|as successor t$',
                   'as successor trustee', words)
    words = re.sub('as t$|as s t$|as sole t$|as tr$|as tru$|as trus$|as trust$|as truste$|as trustee$|as trustee o$|as trustee of$|trustee of$|trustee of$|tr$|tru$|trus$|truste$|trustee$|, t|, tr|, tru|, trus|, trust|, truste',
                   'as trustee', words)
    words = re.sub('su$|suc$|succ$|succe$|succes$|success$|successo$|successor$|as s$|as su$|as suc$|as succ$|as succe$|as sucess$|as successo$|, s$|, su$|, suc$|, succ$|, succe$|, succes$|, success$|, successo$',
                   'as successor', words)

    if re.search(entity_keywords, words) or re.search(r'\d{4}|\d{3}', words) or re.search('as trustee$|as successor$|as successor trustee$', words):
        id = words
        return id

    words = re.sub(' in$|indi$|indiv$|indivi$|indivi$|individ$|individu$|individua$|individual$|not i$|not ind$| ind$| inde$|indep$|indepe$|indepen$|independ$|independe$|independen$|independent$',
                   '', words)

    tokens = split_logic(words)

    id = name_selector(tokens)

    return id


def split_logic(words: str):

    words = re.sub(' +', ' ', words)

    if words.isspace() or re.search(r'^[.]*$', words) or words == 'Empty Name':
        return 'Empty Name'

    words = re.sub(' as$| as $|as $','', words)

    _and = re.search(r'\b and\b|\b an$\b|\b a$\b|f k a|\bfka\b| n k a|\bnka\b|\b aka\b|a k a|\b kna\b|k n a| f k$|n k$|a k$|\b not\b| married', words)

    if _and:
        tokens = words.split(_and.group())
        tokens = tokens[0].strip().split()
    else:
        tokens = words.split()

    return tokens


def name_selector(tokens):
    if tokens == 'Empty Name':
        return tokens

    if tokens[-1] in ['jr', 'sr', 'ii', 'iii', 'iv', 'v']:
        tokens = tokens[:-1]
    if len(tokens) == 2:
        id = tokens[1]
    if len(tokens) == 3:
        id = tokens[2]
    else:
        id = tokens[-1]

    return id


In [5]:
def get_category(row, col: str):

    column = col + '_id'
    words = row[column]

    if re.search(entity_keywords, words):
        return 'legal_entity'
    elif words == 'Empty Name':
        return 'none'
    else:
        return 'person'


In [6]:
def get_role(row, col: str):
    role = None
    column = col + '_id'
    words = row[column]

    suc_trust = re.search(' as successor trustee' , words)
    suc = re.search(' as successor' , words)
    trust = re.search(' as trustee' , words)

    if suc_trust:
        role = suc_trust.group()

    if suc:
        role = suc.group()

    if trust:
        role = trust.group()

    return role


def clean_id(row: object, col: str):
    column = col + '_id'
    words = row[column]

    words = re.sub(r' as successor trustee|\b as successor\b| as trustee', '', words)
    words = re.sub(' as$| as $|as $','', words)

    if not (re.search(entity_keywords, words) or re.search(r'\d{4}|\d{3}', words) or len(words.split()) == 1):
        words = name_selector(split_logic(words))

    return words


In [7]:
def transaction_type(row):
    buyer = row['buyer_category']
    seller = row['seller_category']

    return buyer + '-' + seller


In [8]:
sales['buyer_id'] = sales.apply(get_id, args=('buyer',), axis=1)
sales['seller_id'] = sales.apply(get_id, args=('seller',), axis=1)
sales['buyer_category'] = sales.apply(get_category, args=('buyer',), axis=1)
sales['seller_category'] = sales.apply(get_category, args=('seller',), axis=1)
sales['buyer_role'] = sales.apply(get_role, args=('buyer',), axis=1)
sales['seller_role'] = sales.apply(get_role, args=('seller',), axis=1)
sales['buyer_id'] = sales.apply(clean_id, args=('buyer',), axis=1)
sales['seller_id'] = sales.apply(clean_id, args=('seller',), axis=1)
sales['transaction_type'] = sales.apply(transaction_type, axis=1)

In [9]:
sales.head(50)

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
0,0,29,295,2092050370000,2019.0,2019-05-01 00:00:00+00:00,201000.0,5.303196,96812985.0,1915417094,...,NaN,NaN,NaN,kwenamore,salamacha,person,person,None,None,person-person
1,1,72,205,20311070380000,2019.0,2019-09-01 00:00:00+00:00,215000.0,5.332438,98075139.0,1927355080,...,NaN,NaN,NaN,lopez-ramirez,niles holding llc,person,legal_entity,None,None,person-legal_entity
2,2,24,206,10201120410000,2018.0,2018-07-01 00:00:00+00:00,540000.0,5.732394,97373057.0,1822817046,...,NaN,NaN,NaN,doyle,alongi,person,person,None,None,person-person
3,3,27,203,16183030040000,2022.0,2022-02-11 00:00:00+00:00,595000.0,5.774517,7321053.0,2206001050,...,NaN,NaN,NaN,lee-kim,thierauf,person,person,None,None,person-person
4,4,24,203,10272320440000,2020.0,2020-04-01 00:00:00+00:00,285000.0,5.454845,96592558.0,2019607192,...,NaN,NaN,NaN,jabouri,prgam,person,person,None,None,person-person
5,5,39,203,24081290130000,2022.0,2022-02-28 00:00:00+00:00,165000.0,5.217484,7323370.0,2207407612,...,NaN,NaN,NaN,Empty Name,fredrick,none,person,None,None,none-person
6,6,33,234,15011010390000,2020.0,2020-12-29 00:00:00+00:00,480000.0,5.681241,7127160.0,2102855154,...,NaN,NaN,NaN,curtis,limperis,person,person,None,None,person-person
7,7,72,212,19152070400000,2020.0,2020-03-01 00:00:00+00:00,460000.0,5.662758,98039143.0,2009307291,...,NaN,NaN,NaN,vale properties llc,garcia,legal_entity,person,None,None,legal_entity-person
8,8,24,203,10271020260000,2020.0,2020-10-13 00:00:00+00:00,445000.0,5.648360,7093981.0,2031439185,...,NaN,NaN,NaN,souhlas,Empty Name,person,none,None,None,person-none
9,9,24,203,10271020260000,2019.0,2019-10-01 00:00:00+00:00,235000.0,5.371068,98223289.0,1928845114,...,NaN,NaN,NaN,rb property holdings llc,caullieres,legal_entity,person,None,None,legal_entity-person


In [10]:
people = sales[sales.transaction_type == 'person-person']
people = people[people.seller_id == people.buyer_id]

In [11]:
people

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
171,171,32,206,31122050210000,2020.0,2020-06-01 00:00:00+00:00,415000.0,5.618048,96332477.0,2024006011,...,NaN,NaN,NaN,litchfield,litchfield,person,person,None,None,person-person
181,181,76,203,17311140200000,2018.0,2018-10-01 00:00:00+00:00,135000.0,5.130334,96668939.0,1830957008,...,NaN,NaN,NaN,bucio,bucio,person,person,None,None,person-person
213,213,22,278,9131200070000,2019.0,2019-07-01 00:00:00+00:00,375000.0,5.574031,98371433.0,1918941175,...,-0.014935,2.064800,NaN,moten,moten,person,person,None,None,person-person
331,331,72,205,19013140130000,2019.0,2019-01-01 00:00:00+00:00,40000.0,4.602060,97702440.0,1902213051,...,NaN,NaN,NaN,ortega,ortega,person,person,None,None,person-person
530,530,76,211,17332020370000,2018.0,2018-01-01 00:00:00+00:00,285000.0,5.454845,97371947.0,1803208146,...,NaN,NaN,NaN,munoz,munoz,person,person,None,None,person-person
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217508,217508,24,210,10233050410000,2019.0,2019-11-01 00:00:00+00:00,220000.0,5.342423,97333656.0,1932255220,...,NaN,NaN,NaN,korkes,korkes,person,person,None,None,person-person
217584,217584,28,278,27311060200000,2018.0,2018-11-01 00:00:00+00:00,360000.0,5.556303,97336671.0,1833112032,...,NaN,NaN,NaN,schutt,schutt,person,person,None,None,person-person
217885,217885,71,203,13152340300000,2019.0,2019-05-01 00:00:00+00:00,250000.0,5.397940,96998708.0,1919306102,...,NaN,NaN,NaN,derosas,derosas,person,person,None,None,person-person
218007,218007,72,202,20072240070000,2021.0,2021-09-03 00:00:00+00:00,180000.0,5.255273,7236958.0,2125107443,...,NaN,NaN,NaN,flores,flores,person,person,None,None,person-person


In [12]:
people.outlier_description.value_counts()

Valuation Outlier below the mean             52
Per SQFT Outlier below the mean              13
Per SQFT Outlier above the mean               6
Days since last transaction is above mean     2
Valuation Outlier above the mean              2
Name: outlier_description, dtype: int64

In [13]:
people.primary_outlier.value_counts()

Not Outlier                            1677
Raw Price Outlier                        92
Price/SQFT Outlier                       75
Days Since Last Transaction Outlier      19
Price Change Outlier                     10
Transaction Volatility Outlier            4
Name: primary_outlier, dtype: int64

In [14]:
judicial = sales[(sales.seller_id == 'the judicial sales corporation') | (sales.seller_id == 'intercounty judicial sale')]

In [15]:
judicial

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
23,23,76,211,17314120190000,2018.0,2018-12-01 00:00:00+00:00,221000.0,5.344392,98256241.0,1836134037,...,NaN,NaN,NaN,vt investment corporation,the judicial sales corporation,legal_entity,legal_entity,None,None,legal_entity-legal_entity
25,25,72,205,19362161140000,2019.0,2019-11-01 00:00:00+00:00,78300.0,4.893762,96850892.0,1932906054,...,NaN,NaN,NaN,seem group llc,the judicial sales corporation,legal_entity,legal_entity,None,None,legal_entity-legal_entity
43,43,71,204,13174020190000,2019.0,2019-06-01 00:00:00+00:00,264000.0,5.421604,98482907.0,1919913206,...,NaN,NaN,NaN,pankaj,intercounty judicial sale,person,legal_entity,None,None,person-legal_entity
82,82,24,206,10141110170000,2018.0,2018-06-01 00:00:00+00:00,496000.0,5.695482,96223066.0,1820419271,...,NaN,NaN,NaN,kalmin,the judicial sales corporation,person,legal_entity,None,None,person-legal_entity
115,115,24,203,10141200290000,2019.0,2019-05-01 00:00:00+00:00,234000.0,5.369216,97739970.0,1921049246,...,NaN,NaN,NaN,amp realty llc,the judicial sales corporation,legal_entity,legal_entity,None,None,legal_entity-legal_entity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216049,216049,72,295,25204040610000,2018.0,2018-08-01 00:00:00+00:00,18900.0,4.276462,96520428.0,1823217002,...,NaN,NaN,NaN,smith,the judicial sales corporation,person,legal_entity,None,None,person-legal_entity
216430,216430,13,203,28341040190000,2022.0,2022-01-03 00:00:00+00:00,103471.0,5.014819,7315795.0,2208312036,...,NaN,NaN,NaN,golden city investment inc,the judicial sales corporation,legal_entity,legal_entity,None,None,legal_entity-legal_entity
216819,216819,32,207,31172060270000,2018.0,2018-02-01 00:00:00+00:00,102000.0,5.008600,97366199.0,1806149132,...,NaN,NaN,NaN,north shore holdings ltd,the judicial sales corporation,legal_entity,legal_entity,None,None,legal_entity-legal_entity
217236,217236,11,202,16192140090000,2019.0,2019-06-01 00:00:00+00:00,126100.0,5.100715,97257419.0,1916516035,...,NaN,NaN,NaN,black gold properties lp,the judicial sales corporation,legal_entity,legal_entity,None,None,legal_entity-legal_entity


In [16]:
judicial.primary_outlier.value_counts()

Not Outlier                            1578
Raw Price Outlier                       415
Price/SQFT Outlier                      138
Transaction Volatility Outlier           64
Price Change Outlier                      5
Days Since Last Transaction Outlier       1
Name: primary_outlier, dtype: int64

In [17]:
judicial.outlier_description.value_counts()

Valuation Outlier below the mean        256
Number of transactions is above mean     15
Per SQFT Outlier above the mean           9
Valuation Outlier above the mean          6
Per SQFT Outlier below the mean           5
Price change descreasing from mean        5
Name: outlier_description, dtype: int64

In [18]:
judicial.buyer_id.value_counts().head(35)

vt investment corporation        58
jva il llc                       54
chicago title land trust         39
amp realty llc                   35
wells fargo bank national        34
black gold properties lp         33
north shore holdings ltd         32
cirrus investment group          32
saleem mohammed                  28
carrington mortgage servi        27
illinois reo sales inc           26
stargate investments llc         26
cerberus sfr holdings l          25
midfirst bank                    24
jet acquisitions llc             22
secretary of veterans affairs    21
seem group llc                   21
south bay partners llc           17
pch properties llc               17
lakefront investment grou        16
new age properties llc           16
ohs properties lp                15
real equities inc                14
corona investments llc           14
third coast holdings llc         13
ridgeway group llc               13
ali                              13
nationstar mortgage llc d   

In [19]:
sales.buyer_id.value_counts().head(500).to_dict()

{'Empty Name': 13217,
 'garcia': 1214,
 'rodriguez': 1112,
 'hernandez': 1096,
 'martinez': 1066,
 'gonzalez': 953,
 'williams': 945,
 'johnson': 927,
 'patel': 904,
 'smith': 898,
 'chicago title land trust': 860,
 'lopez': 852,
 'perez': 731,
 'sanchez': 672,
 'jones': 634,
 'ramirez': 626,
 'brown': 622,
 'flores': 537,
 'diaz': 515,
 'davis': 509,
 'torres': 508,
 'jackson': 463,
 'gomez': 447,
 'rivera': 434,
 'cerberus sfr holdings ii': 421,
 'lee': 410,
 'thomas': 409,
 'miller': 389,
 'harris': 371,
 'anderson': 348,
 'reyes': 342,
 'cruz': 338,
 'morales': 327,
 'alvarez': 324,
 'wilson': 322,
 'taylor': 315,
 'guzman': 310,
 'herrera': 304,
 'ortiz': 299,
 'moore': 295,
 'chavez': 281,
 'robinson': 276,
 'nguyen': 274,
 'gutierrez': 273,
 'jimenez': 269,
 'ruiz': 265,
 'munoz': 262,
 'delgado': 256,
 'kim': 251,
 'walker': 249,
 'white': 247,
 'lewis': 246,
 'khan': 244,
 'mendoza': 243,
 'ramos': 239,
 'salgado': 237,
 'martin': 234,
 'moreno': 230,
 'murphy': 229,
 'shah': 

In [20]:
sales.seller_id.value_counts().head(500).to_dict()

{'Empty Name': 30949,
 'chicago title land trust': 3871,
 'the judicial sales corporation': 1771,
 'fannie mae': 1730,
 'us bank national association': 1089,
 'secretary of housing and urban development': 809,
 'johnson': 686,
 'smith': 679,
 'federal home loan mortgag': 678,
 'garcia': 552,
 'wells fargo bank national': 529,
 'patel': 528,
 'first midwest bank': 514,
 'williams': 505,
 'rodriguez': 466,
 'deutsche bank national trust company': 463,
 'martinez': 450,
 'hernandez': 436,
 'gonzalez': 431,
 'intercounty judicial sale': 430,
 'brown': 426,
 'jones': 424,
 'the bank of new york mell': 370,
 'wilmington savings fund s': 359,
 'lopez': 334,
 'miller': 334,
 'lee': 327,
 'davis': 322,
 'perez': 320,
 'grandview capital llc': 311,
 'thomas': 297,
 'anderson': 289,
 'sanchez': 280,
 'ramirez': 273,
 'marquette bank': 270,
 'diaz': 252,
 'bank of america national': 242,
 'harris': 241,
 'the chicago trust company': 238,
 'jackson': 233,
 'flores': 228,
 'trustee': 226,
 'taylor':

In [21]:
sales[sales.seller_id == 'co-'].tail(25)

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
90111,90111,38,278,3171180030000,2018.0,2018-06-01 00:00:00+00:00,675000.0,5.829304,96668126.0,1818329378,...,NaN,NaN,NaN,taranda,co-,person,person,None,None,person-person
109402,109402,18,234,6351150270000,2019.0,2019-04-01 00:00:00+00:00,252000.0,5.401401,98073893.0,1916141059,...,NaN,NaN,NaN,lemus,co-,person,person,None,None,person-person
113525,113525,29,278,2181040090000,2018.0,2018-08-01 00:00:00+00:00,710000.0,5.851258,98078554.0,1829549077,...,NaN,NaN,NaN,serpico,co-,person,person,None,None,person-person
144594,144594,71,203,13171160230000,2019.0,2019-05-01 00:00:00+00:00,293000.0,5.466868,96367537.0,1912346188,...,NaN,NaN,NaN,guanio,co-,person,person,None,None,person-person
153446,153446,25,278,4251001080000,2019.0,2019-06-01 00:00:00+00:00,640000.0,5.806180,97369478.0,1916908057,...,NaN,NaN,NaN,wynn,co-,person,person,None,None,person-person
156851,156851,38,207,3311120370000,2020.0,2020-02-01 00:00:00+00:00,420000.0,5.623249,96476169.0,2007155332,...,NaN,NaN,NaN,lambert,co-,person,person,as trustee,None,person-person
175084,175084,35,278,7223100170000,2019.0,2019-02-01 00:00:00+00:00,420000.0,5.623249,96441638.0,1906634075,...,NaN,NaN,NaN,williams,co-,person,person,None,None,person-person
176349,176349,73,206,14203020140000,2020.0,2020-07-01 00:00:00+00:00,3350000.0,6.525045,97259696.0,2026007152,...,1.025221,2.621652,Valuation Outlier above the mean,chicago title land trust,co-,legal_entity,person,None,None,legal_entity-person
194648,194648,71,211,13132130300000,2020.0,2020-08-01 00:00:00+00:00,460000.0,5.662758,96481815.0,2029121218,...,NaN,NaN,NaN,p3 properties llc - 2522,co-,legal_entity,person,None,None,legal_entity-person
208882,208882,22,203,9111000050000,2018.0,2018-04-01 00:00:00+00:00,245000.0,5.389166,97446693.0,1811519089,...,NaN,NaN,NaN,filip,co-,person,person,None,None,person-person


In [22]:
sales[sales.seller_id == 'of']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
7049,7049,71,205,13114290060000,2021.0,2021-10-06 00:00:00+00:00,850000.0,5.929419,7251177.0,2128616327,...,-1.929744,2.417435,NaN,lin,of,person,person,None,None,person-person
10359,10359,31,204,15293260150000,2018.0,2018-03-01 00:00:00+00:00,305000.0,5.484300,97409383.0,1810201049,...,NaN,NaN,NaN,garcia,of,person,person,None,None,person-person
12491,12491,72,202,20184110450000,2020.0,2020-07-01 00:00:00+00:00,38000.0,4.579784,97816872.0,2023840135,...,NaN,NaN,NaN,ochoa,of,person,person,None,None,person-person
12845,12845,25,203,4162130020000,2018.0,2018-05-01 00:00:00+00:00,362500.0,5.559308,96185179.0,1813649129,...,NaN,NaN,NaN,konsor enterprise inc,of,legal_entity,person,None,None,legal_entity-person
19141,19141,70,205,20224150270000,2020.0,2020-02-01 00:00:00+00:00,68000.0,4.832509,97513222.0,2004915004,...,NaN,NaN,NaN,chicago title land trust,of,legal_entity,person,None,None,legal_entity-person
26651,26651,77,211,17081080300000,2021.0,2021-09-30 00:00:00+00:00,1280000.0,6.107210,7257610.0,2130020143,...,NaN,NaN,NaN,Empty Name,of,none,person,None,None,none-person
26826,26826,71,202,13211180270000,2020.0,2020-03-01 00:00:00+00:00,243500.0,5.386499,96963692.0,2008057034,...,NaN,NaN,NaN,rodea,of,person,person,None,None,person-person
26934,26934,38,203,3204120010000,2020.0,2020-02-01 00:00:00+00:00,292000.0,5.465383,97440383.0,2009107334,...,NaN,NaN,NaN,flaherty,of,person,person,None,None,person-person
41302,41302,72,203,19103150070000,2020.0,2020-05-01 00:00:00+00:00,190000.0,5.278754,96369412.0,2019507066,...,NaN,NaN,NaN,lopez,of,person,person,None,None,person-person
43733,43733,13,234,28212140200000,2019.0,2019-05-01 00:00:00+00:00,185000.0,5.267172,97925283.0,1915618020,...,NaN,NaN,NaN,gonzales,of,person,person,None,None,person-person


In [23]:
sales[sales.seller_id == 'j']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
801,801,29,295,2361130390000,2019.0,2019-07-01 00:00:00+00:00,303000.0,5.481443,97963571.0,1922655171,...,NaN,NaN,NaN,gonzales,j,person,person,None,None,person-person
1254,1254,29,202,2031050080000,2018.0,2018-02-01 00:00:00+00:00,436000.0,5.639486,98330505.0,1806049162,...,-1.378293,1.577405,NaN,j,j,person,person,None,None,person-person
12915,12915,71,203,13193250500000,2018.0,2018-09-01 00:00:00+00:00,290000.0,5.462398,96816701.0,1825719062,...,NaN,NaN,NaN,arellano,j,person,person,None,None,person-person
17011,17011,35,234,7284050430000,2019.0,2019-08-01 00:00:00+00:00,356000.0,5.551450,97332896.0,1923355228,...,NaN,NaN,NaN,ortiz,j,person,person,None,None,person-person
17064,17064,72,202,19123140100000,2018.0,2018-08-01 00:00:00+00:00,42000.0,4.623249,96520483.0,1825557085,...,NaN,NaN,NaN,gk home inc,j,legal_entity,person,None,None,legal_entity-person
27866,27866,77,211,16262220150000,2018.0,2018-10-01 00:00:00+00:00,75000.0,4.875061,98412161.0,1830546046,...,-1.994826,2.197478,NaN,gams properties llc,j,legal_entity,person,None,None,legal_entity-person
28816,28816,36,204,19333160060000,2020.0,2020-09-03 00:00:00+00:00,233900.0,5.369030,7095261.0,2030646445,...,NaN,NaN,NaN,viloria,j,person,person,None,None,person-person
31717,31717,24,203,10201220710000,2020.0,2020-01-01 00:00:00+00:00,160000.0,5.204120,96177324.0,2004210066,...,-0.380966,1.361083,NaN,bochenko,j,person,person,None,None,person-person
31807,31807,39,278,24011290370000,2021.0,2021-03-25 00:00:00+00:00,365000.0,5.562293,7204481.0,2115420106,...,NaN,NaN,NaN,brown,j,person,person,None,None,person-person
49097,49097,28,234,27234110110000,2018.0,2018-10-01 00:00:00+00:00,258000.0,5.411620,97930384.0,1828419061,...,NaN,NaN,NaN,lucas,j,person,person,None,None,person-person


In [24]:
sales[sales.seller_id == 't'].head(50)

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
7581,7581,16,295,8294150060000,2018.0,2018-04-01 00:00:00+00:00,258000.0,5.411620,97039077.0,1811501087,...,NaN,NaN,NaN,mcinerney,t,person,person,None,None,person-person
8475,8475,39,202,24171210220000,2018.0,2018-06-01 00:00:00+00:00,152000.0,5.181844,97410021.0,1817204043,...,NaN,NaN,NaN,caltagerone,t,person,person,None,None,person-person
12700,12700,13,234,28163100070000,2018.0,2018-03-01 00:00:00+00:00,169000.0,5.227887,97113388.0,1808947151,...,NaN,NaN,NaN,farrell,t,person,person,None,None,person-person
12955,12955,71,203,13091160240000,2018.0,2018-02-01 00:00:00+00:00,310000.0,5.491362,97113292.0,1805734029,...,NaN,NaN,NaN,gocha,t,person,person,None,None,person-person
13764,13764,13,203,28203000070000,2019.0,2019-02-01 00:00:00+00:00,230000.0,5.361728,96144384.0,1905341063,...,NaN,NaN,NaN,thurbush,t,person,person,None,None,person-person
14704,14704,29,295,2222070360000,2018.0,2018-01-01 00:00:00+00:00,400000.0,5.602060,97113057.0,1801149087,...,NaN,NaN,NaN,donile,t,person,person,None,None,person-person
18576,18576,37,203,29092040430000,2019.0,2019-05-01 00:00:00+00:00,77000.0,4.886491,96626837.0,1914349275,...,NaN,NaN,NaN,mcintosh,t,person,person,None,None,person-person
19066,19066,30,203,23032090420000,2021.0,2021-05-21 00:00:00+00:00,265000.0,5.423246,7203704.0,2118003204,...,NaN,NaN,NaN,knapp,t,person,person,None,None,person-person
19278,19278,24,205,10152100490000,2019.0,2019-08-01 00:00:00+00:00,410000.0,5.612784,97926744.0,1923555084,...,NaN,NaN,NaN,khan,t,person,person,None,None,person-person
19588,19588,76,211,17282110170000,2018.0,2018-11-01 00:00:00+00:00,670000.0,5.826075,96557943.0,1831855010,...,-2.042376,0.558806,Per SQFT Outlier above the mean,229 w alexander llc,t,legal_entity,person,None,None,legal_entity-person


In [25]:
sales[sales.seller_id == 'e']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
857,857,31,210,15031300150000,2018.0,2018-07-01 00:00:00+00:00,100000.0,5.000000,98330596.0,1819855111,...,NaN,NaN,NaN,pilati,e,person,person,None,None,person-person
7474,7474,29,211,2121000650000,2018.0,2018-01-01 00:00:00+00:00,475000.0,5.676694,96513265.0,1801255067,...,NaN,NaN,NaN,select inv fund llc,e,legal_entity,person,None,None,legal_entity-person
8716,8716,72,205,20074100040000,2019.0,2019-01-01 00:00:00+00:00,80000.0,4.903090,96478438.0,1902249088,...,NaN,NaN,NaN,summer lakes llc,e,legal_entity,person,None,None,legal_entity-person
22223,22223,29,204,2282090170000,2018.0,2018-01-01 00:00:00+00:00,335000.0,5.525045,97668841.0,1803615050,...,NaN,NaN,NaN,ohm,e,person,person,None,None,person-person
26767,26767,71,203,13073130520000,2018.0,2018-06-01 00:00:00+00:00,210000.0,5.322219,98152156.0,1819118012,...,NaN,NaN,NaN,bcl-home rehab sub 1,e,legal_entity,person,None,None,legal_entity-person
44708,44708,70,211,20153070250000,2019.0,2019-06-01 00:00:00+00:00,95000.0,4.977724,97443968.0,1917108063,...,NaN,NaN,NaN,gbaf llc,e,legal_entity,person,None,None,legal_entity-person
52635,52635,25,203,4353240070000,2018.0,2018-05-01 00:00:00+00:00,305000.0,5.484300,98077496.0,1812429071,...,NaN,NaN,NaN,dubray,e,person,person,None,None,person-person
53329,53329,73,211,14191200190000,2019.0,2019-09-01 00:00:00+00:00,575000.0,5.759668,97741015.0,1924934018,...,NaN,NaN,NaN,3909 n seeley llc,e,legal_entity,person,None,None,legal_entity-person
59108,59108,27,206,16181040100000,2018.0,2018-07-01 00:00:00+00:00,592000.0,5.772322,97447336.0,1820419215,...,NaN,NaN,NaN,puntillo,e,person,person,None,None,person-person
64204,64204,72,234,24133110480000,2020.0,2020-11-06 00:00:00+00:00,385000.0,5.585461,7116890.0,2102021302,...,NaN,NaN,NaN,kevin,e,person,person,None,None,person-person


In [26]:
sales[sales.seller_id == 'r']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
15164,15164,25,278,4344131010000,2021.0,2021-03-31 00:00:00+00:00,1095000.0,6.039414,7169604.0,2110212324,...,NaN,NaN,NaN,desmond,r,person,person,None,None,person-person
39457,39457,16,203,8323080220000,2019.0,2019-04-01 00:00:00+00:00,240000.0,5.380211,97888726.0,1912733352,...,NaN,NaN,NaN,jarmula,r,person,person,None,None,person-person
50088,50088,37,203,30311010070000,2020.0,2020-12-23 00:00:00+00:00,93500.0,4.970812,7156309.0,2107128019,...,NaN,NaN,NaN,arenas,r,person,person,None,None,person-person
52280,52280,29,202,2264160030000,2018.0,2018-05-01 00:00:00+00:00,214900.0,5.332236,98336882.0,1815825047,...,NaN,NaN,NaN,schulke,r,person,person,None,None,person-person
74637,74637,23,206,5271010150000,2018.0,2018-08-01 00:00:00+00:00,1659600.0,6.220003,96371827.0,1822710012,...,NaN,NaN,NaN,laroyia,r,person,person,None,None,person-person
78957,78957,35,278,7213000280000,2019.0,2019-12-01 00:00:00+00:00,415000.0,5.618048,96331775.0,1935455070,...,NaN,NaN,NaN,baba,r,person,person,None,None,person-person
81090,81090,37,203,30311010080000,2020.0,2020-12-23 00:00:00+00:00,93500.0,4.970812,7156310.0,2107128019,...,NaN,NaN,NaN,arenas,r,person,person,None,None,person-person
87286,87286,71,203,12243110340000,2019.0,2019-06-01 00:00:00+00:00,323000.0,5.509203,96775789.0,1918933015,...,NaN,NaN,NaN,butler,r,person,person,None,None,person-person
91517,91517,15,211,16281270240000,2019.0,2019-10-01 00:00:00+00:00,220000.0,5.342423,97630024.0,1929155020,...,NaN,NaN,NaN,servin,r,person,person,None,None,person-person
92588,92588,22,203,9351190110000,2021.0,2021-02-21 00:00:00+00:00,407100.0,5.609701,7151747.0,2106817050,...,NaN,NaN,NaN,Empty Name,r,none,person,None,None,none-person


In [27]:
sales[sales.seller_id == 'd']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
456,456,31,204,15274100690000,2019.0,2019-08-01 00:00:00+00:00,265000.0,5.423246,96590513.0,1922146235,...,NaN,NaN,NaN,sahagun-zavala,d,person,person,None,None,person-person
5710,5710,18,203,6361200380000,2018.0,2018-02-01 00:00:00+00:00,170000.0,5.230449,96624433.0,1806049160,...,NaN,NaN,NaN,m,d,person,person,None,None,person-person
17362,17362,73,211,14291010110000,2018.0,2018-03-01 00:00:00+00:00,925000.0,5.966142,97260829.0,1807347017,...,NaN,NaN,NaN,marks-michaelis family re,d,legal_entity,person,None,None,legal_entity-person
20436,20436,34,203,15261280060000,2018.0,2018-09-01 00:00:00+00:00,215000.0,5.332438,97744597.0,1826806175,...,NaN,NaN,NaN,hernandez,d,person,person,None,None,person-person
20909,20909,38,205,3314030200000,2018.0,2018-01-01 00:00:00+00:00,300000.0,5.477121,96926927.0,1806849061,...,NaN,NaN,NaN,houston,d,person,person,None,None,person-person
28885,28885,71,211,13062140290000,2018.0,2018-04-01 00:00:00+00:00,476000.0,5.677607,97632276.0,1815829355,...,NaN,NaN,NaN,sheridan,d,person,person,None,None,person-person
34059,34059,70,207,26173120140000,2019.0,2019-03-01 00:00:00+00:00,210000.0,5.322219,98407438.0,1907257108,...,NaN,NaN,NaN,salinas,d,person,person,None,None,person-person
34186,34186,70,295,25031250140000,2020.0,2020-11-19 00:00:00+00:00,100000.0,5.000000,7114875.0,2102617525,...,NaN,NaN,NaN,taitt,d,person,person,None,None,person-person
38755,38755,18,203,6182110230000,2020.0,2020-03-01 00:00:00+00:00,167000.0,5.222716,98002267.0,2010520045,...,-2.354546,2.031138,NaN,hernandez,d,person,person,None,None,person-person
39902,39902,20,203,12323240030000,2018.0,2018-02-01 00:00:00+00:00,226000.0,5.354108,97588840.0,1806549049,...,NaN,NaN,NaN,e,d,person,person,None,None,person-person


In [28]:
sales[sales.seller_id == 'k']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
3443,3443,72,211,20171130370000,2018.0,2018-12-01 00:00:00+00:00,75000.0,4.875061,97590934.0,1901122025,...,NaN,NaN,NaN,kinsman,k,person,person,None,None,person-person
25239,25239,39,203,24043010380000,2019.0,2019-09-01 00:00:00+00:00,203000.0,5.307496,96590914.0,1925208211,...,NaN,NaN,NaN,kruas,k,person,person,None,None,person-person
26804,26804,71,203,13243170140000,2019.0,2019-08-01 00:00:00+00:00,574000.0,5.758912,96664669.0,1922722045,...,NaN,NaN,NaN,piehl,k,person,person,None,None,person-person
37395,37395,39,203,24043010370000,2019.0,2019-09-01 00:00:00+00:00,203000.0,5.307496,96627951.0,1925208211,...,NaN,NaN,NaN,kruas,k,person,person,None,None,person-person
42400,42400,77,211,13251020310000,2020.0,2020-06-01 00:00:00+00:00,598500.0,5.777064,98038849.0,2021121099,...,NaN,NaN,NaN,dill,k,person,person,None,None,person-person
63161,63161,29,234,2032090070000,2018.0,2018-03-01 00:00:00+00:00,275000.0,5.439333,98145400.0,1809257003,...,NaN,NaN,NaN,lauren,k,person,person,None,as trustee,person-person
177453,177453,71,203,13232290030000,2018.0,2018-07-01 00:00:00+00:00,329000.0,5.517196,96890751.0,1819949168,...,NaN,NaN,NaN,kwak,k,person,person,None,None,person-person
201976,201976,16,203,8282160140000,2019.0,2019-06-01 00:00:00+00:00,185000.0,5.267172,98407896.0,1918346072,...,NaN,NaN,NaN,hunter,k,person,person,None,None,person-person


In [29]:
sales[sales.seller_id == 's']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
1683,1683,72,205,25063020540000,2019.0,2019-06-01 00:00:00+00:00,305000.0,5.484300,96887198.0,1918645092,...,NaN,NaN,NaN,hagans,s,person,person,None,None,person-person
4070,4070,39,204,24312010080000,2018.0,2018-12-01 00:00:00+00:00,235000.0,5.371068,96144013.0,1900813011,...,NaN,NaN,NaN,il,s,person,person,None,None,person-person
6835,6835,11,203,16314120380000,2019.0,2019-10-01 00:00:00+00:00,388000.0,5.588832,96257356.0,1929555019,...,NaN,NaN,NaN,claudio,s,person,person,None,None,person-person
8982,8982,71,203,13053240420000,2018.0,2018-07-01 00:00:00+00:00,340000.0,5.531479,96965236.0,1820533019,...,NaN,NaN,NaN,le,s,person,person,None,None,person-person
12882,12882,71,203,13112210030000,2018.0,2018-11-01 00:00:00+00:00,260000.0,5.414973,96261081.0,1833847040,...,NaN,NaN,NaN,patel,s,person,person,None,None,person-person
18796,18796,10,208,1013080290000,2020.0,2020-06-01 00:00:00+00:00,865000.0,5.937016,98446693.0,2025407171,...,NaN,NaN,NaN,wonder,s,person,person,None,None,person-person
19243,19243,22,234,9134120140000,2018.0,2018-03-01 00:00:00+00:00,285000.0,5.454845,97849259.0,1808949147,...,NaN,NaN,NaN,bijay,s,person,person,None,as trustee,person-person
22012,22012,34,203,15251070190000,2018.0,2018-04-01 00:00:00+00:00,215000.0,5.332438,97335316.0,1812901227,...,NaN,NaN,NaN,wright,s,person,person,None,None,person-person
23602,23602,37,203,30072080090000,2020.0,2020-02-01 00:00:00+00:00,70000.0,4.845098,98372659.0,2007057031,...,NaN,NaN,NaN,carrasco,s,person,person,None,None,person-person
31312,31312,72,205,20064230150000,2019.0,2019-11-01 00:00:00+00:00,70000.0,4.845098,97742030.0,1931246020,...,NaN,NaN,NaN,chagoya,s,person,person,None,None,person-person


In [30]:
sales[sales.seller_id == 'a']

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
2666,2666,13,234,28321000200000,2019.0,2019-03-01 00:00:00+00:00,265000.0,5.423246,97035429.0,1906506100,...,-1.479577,0.635400,NaN,thompson,a,person,person,None,None,person-person
24961,24961,29,204,2354070040000,2018.0,2018-02-01 00:00:00+00:00,396000.0,5.597695,97594685.0,1806549074,...,NaN,NaN,NaN,baar,a,person,person,None,None,person-person
31353,31353,35,234,7312160190000,2019.0,2019-03-01 00:00:00+00:00,150000.0,5.176091,97443062.0,1906549210,...,-0.446444,1.611833,NaN,little sumpin renovations,a,legal_entity,person,None,None,legal_entity-person
34590,34590,18,278,6062090030000,2020.0,2020-02-01 00:00:00+00:00,319999.0,5.505149,97329113.0,2012608141,...,NaN,NaN,NaN,mendoza,a,person,person,None,None,person-person
52146,52146,19,278,22283050040000,2019.0,2019-08-01 00:00:00+00:00,425000.0,5.628389,97555226.0,1924916006,...,NaN,NaN,NaN,popadic,a,person,person,None,None,person-person
62007,62007,29,203,2361030080000,2019.0,2019-05-01 00:00:00+00:00,235000.0,5.371068,96886346.0,1915455004,...,NaN,NaN,NaN,chad,a,person,person,None,None,person-person
62035,62035,29,207,2261140210000,2018.0,2018-04-01 00:00:00+00:00,282000.0,5.450249,98262140.0,1811655045,...,NaN,NaN,NaN,m,a,person,person,None,None,person-person
72101,72101,13,203,28234010390000,2019.0,2019-09-01 00:00:00+00:00,152000.0,5.181844,97296075.0,1928055342,...,NaN,NaN,NaN,goston,a,person,person,None,None,person-person
89774,89774,35,295,7184030810000,2020.0,2020-01-01 00:00:00+00:00,147000.0,5.167317,96400453.0,2004413066,...,NaN,NaN,NaN,divorce,a,person,person,None,None,person-person
112097,112097,74,295,17043160470000,2018.0,2018-09-01 00:00:00+00:00,535000.0,5.728354,98115563.0,1826442070,...,NaN,NaN,NaN,mueller,a,person,person,None,None,person-person


In [31]:
sales[sales.buyer_id == 't'].head(20)

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
4349,4349,30,204,23024000280000,2018.0,2018-03-01 00:00:00+00:00,217000.0,5.336460,96476311.0,1808246105,...,NaN,NaN,NaN,t,atg trust co tr 000000l016080,person,legal_entity,None,None,person-legal_entity
17571,17571,73,278,14203210070000,2018.0,2018-06-01 00:00:00+00:00,2040000.0,6.309630,96371469.0,1819016051,...,NaN,NaN,NaN,t,1443 henderson llc,person,legal_entity,None,None,person-legal_entity
18203,18203,12,234,32171180150000,2018.0,2018-08-01 00:00:00+00:00,65000.0,4.812913,97856564.0,1825346007,...,-2.112378,0.815801,NaN,t,heerwagen,person,person,None,None,person-person
20139,20139,12,203,32041050120000,2018.0,2018-03-01 00:00:00+00:00,160000.0,5.204120,96772470.0,1808546189,...,NaN,NaN,NaN,t,ca homes prop llc,person,legal_entity,None,None,person-legal_entity
53078,53078,19,295,22272030420000,2020.0,2020-07-01 00:00:00+00:00,296000.0,5.471292,97075183.0,2025515000,...,NaN,NaN,NaN,t,evenhouse,person,person,None,None,person-person
55066,55066,72,203,20041260510000,2020.0,2020-07-01 00:00:00+00:00,339000.0,5.530200,96926062.0,2026607006,...,NaN,NaN,NaN,t,hughes,person,person,None,as trustee,person-person
58684,58684,11,205,16314230390000,2018.0,2018-01-01 00:00:00+00:00,250000.0,5.397940,98076951.0,1803108254,...,NaN,NaN,NaN,t,snezana,person,person,None,None,person-person
61379,61379,29,234,2302030100000,2019.0,2019-06-01 00:00:00+00:00,525000.0,5.720159,98296302.0,1919306021,...,-0.507376,1.372804,Per SQFT Outlier below the mean,t,hughes,person,person,None,None,person-person
61717,61717,77,211,17193080320000,2018.0,2018-01-01 00:00:00+00:00,350000.0,5.544068,97446316.0,1802647116,...,NaN,NaN,NaN,t,chicago title land trust,person,legal_entity,None,None,person-legal_entity
66604,66604,39,203,24284050110000,2018.0,2018-05-01 00:00:00+00:00,213000.0,5.328380,97886172.0,1817347005,...,NaN,NaN,NaN,t,noftz,person,person,None,None,person-person


In [32]:
sales[sales.buyer_id == 'a'].head(20)

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
26100,26100,31,204,15343060280000,2019.0,2019-12-01 00:00:00+00:00,430000.0,5.633468,96888064.0,1935208118,...,NaN,NaN,NaN,a,filbey,person,person,None,None,person-person
45878,45878,22,203,9224130390000,2021.0,2021-01-18 00:00:00+00:00,549000.0,5.739572,7136855.0,2104007167,...,NaN,NaN,NaN,a,robert,person,person,None,None,person-person
56940,56940,28,278,27141050120000,2019.0,2019-09-01 00:00:00+00:00,360000.0,5.556303,96591567.0,1926046341,...,NaN,NaN,NaN,a,wood,person,person,None,None,person-person
70021,70021,19,234,22283070070000,2019.0,2019-04-01 00:00:00+00:00,375000.0,5.574031,96775290.0,1911908002,...,NaN,NaN,NaN,a,chicago title land trust,person,legal_entity,None,None,person-legal_entity
76884,76884,25,204,4322090400000,2020.0,2020-01-01 00:00:00+00:00,682334.0,5.833997,98478608.0,2004422105,...,NaN,NaN,NaN,a,hawthorn 45 llc,person,legal_entity,None,None,person-legal_entity
77929,77929,20,203,12211030960000,2018.0,2018-12-01 00:00:00+00:00,220000.0,5.342423,97448094.0,1835349231,...,NaN,NaN,NaN,a,karl,person,person,None,None,person-person
114681,114681,26,204,12133080100000,2020.0,2020-09-01 00:00:00+00:00,375000.0,5.574031,97483189.0,2030016029,...,NaN,NaN,NaN,a,regacho,person,person,None,None,person-person


In [33]:
sales[sales.buyer_id == 'c'].head(20)

,Unnamed: 0,township_code,class,pin,year,sale_date,sale_price,sale_price_log10,sale_key,doc_no,...,outlier_std_lower,outlier_std_upper,outlier_description,buyer_id,seller_id,buyer_category,seller_category,buyer_role,seller_role,transaction_type
2496,2496,17,206,5334160180000,2020.0,2020-08-01 00:00:00+00:00,770000.0,5.886491,96667009.0,2028739079,...,NaN,NaN,NaN,c,sawczuk,person,person,None,None,person-person
20943,20943,30,234,23142020180000,2018.0,2018-02-01 00:00:00+00:00,238000.0,5.376577,96958377.0,1807457162,...,NaN,NaN,NaN,c,marc,person,person,None,None,person-person
24801,24801,38,203,3331180070000,2021.0,2021-01-05 00:00:00+00:00,296000.0,5.471292,7142523.0,2103601285,...,NaN,NaN,NaN,c,weatherred,person,person,None,None,person-person
54472,54472,29,278,2352130070000,2018.0,2018-09-01 00:00:00+00:00,453000.0,5.656098,96557635.0,1828222041,...,NaN,NaN,NaN,c,vida,person,person,None,None,person-person
69363,69363,71,211,13054290240000,2018.0,2018-04-01 00:00:00+00:00,445000.0,5.648360,96704692.0,1811534065,...,NaN,NaN,NaN,c,oliveri,person,person,None,None,person-person
100288,100288,73,206,14283120650000,2019.0,2019-08-01 00:00:00+00:00,2440000.0,6.387390,98259873.0,1925555060,...,1.025221,2.621652,Valuation Outlier above the mean,c,hoyer,person,person,None,None,person-person
127130,127130,29,278,2092030280000,2020.0,2020-07-01 00:00:00+00:00,450000.0,5.653213,97964579.0,2024439051,...,NaN,NaN,NaN,c,blue flag holdings llc,person,legal_entity,None,None,person-legal_entity
132235,132235,74,208,14331280100000,2018.0,2018-07-01 00:00:00+00:00,3778300.0,6.577296,97707277.0,1822216035,...,NaN,NaN,NaN,c,murphy,person,person,None,None,person-person
145784,145784,28,295,27212030250000,2018.0,2018-11-01 00:00:00+00:00,315000.0,5.498311,97485325.0,1833208077,...,NaN,NaN,NaN,c,dytrych,person,person,None,None,person-person
171243,171243,10,234,1012120160000,2019.0,2019-03-01 00:00:00+00:00,453000.0,5.656098,97368202.0,1906512064,...,NaN,NaN,NaN,c,jackson,person,person,None,None,person-person


## Validation

In [34]:
valid = sales[['buyer_name','seller_name','buyer_id','seller_id']]

In [35]:
pd.set_option('display.max_rows', 500)
valid.sample(200)


,buyer_name,seller_name,buyer_id,seller_id
176317,LENORE JOSEPH,ARNOLD VANDER NAT TRUSTEE,joseph,nat
136567,GABOR TALAN,GEORGE M DLUHY AND SUSAN,talan,dluhy
213667,NATALIE DOMBI,MCCORMICK HOMES LLC,dombi,mccormick homes llc
109703,LESTER TAYLOR,,taylor,Empty Name
211427,MICHELLE ROSALES,KATARZYNA KUC,michelle rosales,kuc
30552,CHICAGO ANTI-EVICTION CAM,FEDERAL NATIONAL MORTGAGE,cam,fannie mae
184219,ASHANTA ROSS,,ross,Empty Name
99211,DENEAN AND DANIA MANAGEME,GLENDALE DOOLEY,denean,dooley
82173,REYES HOME REMODELING IN,SNL INVESTMENTS LLC,reyes home remodeling in,snl investments llc
108259,SUSANA GARIBAY,PETER D WELLS,garibay,wells


In [36]:
(400-8)/400

0.98

In [37]:
(400-5)/400

0.9875

In [38]:
(400-7)/400

0.9825